In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Data overview

In [ ]:
# importing library
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,f1_score
from xgboost import XGBClassifier


In [ ]:
# loading data
df_application = pd.read_csv('/kaggle/input/credit-card-approval-prediction/application_record.csv')
df_credit = pd.read_csv('/kaggle/input/credit-card-approval-prediction/credit_record.csv')

In [ ]:
df_application.sample(5)

In [ ]:
df_credit.sample(5)

In [ ]:
df_application.info()

In [ ]:
df_credit.info()

In [ ]:
df_application.shape

In [ ]:
df_credit.shape

In [ ]:
# df = pd.merge(df_application, df_credit, on='ID',how='left')
# mreging data
df = pd.merge(df_application, df_credit, on='ID', how='inner')

In [ ]:
df

In [ ]:
print('Application Record data shape: ',df_application.shape)
print('Credit Record data shape: ',df_credit.shape)
print('Merged data shape: ',df.shape)

In [ ]:
df.info()

In [ ]:
# checking duplicate value
df.duplicated().sum()

In [ ]:
# checking null value
df.isnull().sum()

In [ ]:
df.isnull().sum()/df.shape[0]*100

In [ ]:
df["OCCUPATION_TYPE"].value_counts()

In [ ]:
df["OCCUPATION_TYPE"].unique()

In [ ]:
# dropping null value 
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
# checking data after dropping null value
df.info()

In [ ]:
df.describe().T.style.background_gradient(cmap='YlOrRd')

# EDA

In [ ]:
# correlation matrix
plt.figure(figsize=(25,8))
sns.heatmap(df.select_dtypes(exclude='object').corr(),annot=True)

In [ ]:
df["CODE_GENDER"].value_counts()

In [ ]:
Categorical=df.select_dtypes(include='object').columns.tolist()
Categorical

In [ ]:
df["FLAG_OWN_CAR"].value_counts()

In [ ]:
# df["FLAG_OWN_CAR"].value_counts()

In [ ]:
fig, axes = plt.subplots(1,3)
colors = ['#ff9999', '#66b3ff']

g1= df['CODE_GENDER'].value_counts().plot.pie(explode=[0,0.1], ax=axes[0], colors=colors, shadow=True)
g1.set_title("Customer Distribution by Gender")

g2= df['FLAG_OWN_CAR'].value_counts().plot.pie(explode=[0,0.1], ax=axes[1],   colors=colors, shadow=True)
g2.set_title("Car Ownership")

g3= df['FLAG_OWN_REALTY'].value_counts().plot.pie(explode=[0,0.1], ax=axes[2],   colors=colors, shadow=True)
g3.set_title("Realty Ownership")

fig.set_size_inches(14,5)

plt.tight_layout()

plt.show()

In [ ]:
# plt.figure(figsize=(6,6))
# colors = ['#ff9999', '#66b3ff']
# explode = (0, 0.1)
# plt.pie(x=df['CODE_GENDER'].value_counts().values, labels=['Female', 'Male'], 
#         autopct='%.1f%%', startangle=90, explode=explode, colors=colors, shadow=True)
# plt.legend(title="Gender", loc="upper right")
# plt.title("Gender Distribution")
# plt.show()

In [ ]:


# plt.figure(figsize=(6,6))
# colors = ['#ff9999', '#66b3ff']
# explode = (0, 0.1)
# plt.pie(x=df["FLAG_OWN_CAR"].value_counts().values, labels=['YES', 'NO'], 
#         autopct='%.1f%%', startangle=90, explode=explode, colors=colors, shadow=True)
# plt.legend(title="Having Car", loc="upper right")
# plt.title("Having Car Distribution")
# plt.show()

In [ ]:
# df["FLAG_OWN_REALTY"].value_counts()

In [ ]:
# sns.countplot (x='FLAG_OWN_REALTY',data=df)
# plt.xticks([0,1],['Y','N'])

In [ ]:
df["NAME_INCOME_TYPE"].value_counts()

In [ ]:
plt.bar(df["NAME_INCOME_TYPE"].value_counts().index, df["NAME_INCOME_TYPE"].value_counts().values, color='Purple')



# Add labels and title
plt.xlabel("Income Type")
plt.ylabel("Count")
plt.title("Value Counts of NAME_INCOME_TYPE")

# Rotate x-axis labels for better visibility
plt.xticks(rotation=45)

# Show plot
plt.show()

In [ ]:
df["NAME_EDUCATION_TYPE"].value_counts()

In [ ]:
plt.bar(df["NAME_EDUCATION_TYPE"].value_counts().index, df["NAME_EDUCATION_TYPE"].value_counts().values,color='green')
# Add labels and title
plt.xlabel("EDUCATION Type")
plt.ylabel("Count")
plt.title("Value Counts of NAME_EDUCATION_TYPE")

# Rotate x-axis labels for better visibility
plt.xticks(rotation=30)

# Show plot
plt.show()

In [ ]:
df["NAME_FAMILY_STATUS"].value_counts()

In [ ]:
plt.figure(figsize=(10, 8))

sns.barplot(data=df,
            y=df["NAME_FAMILY_STATUS"].value_counts().index,
            x=df["NAME_FAMILY_STATUS"].value_counts().values,
            palette='rocket')

In [ ]:
df["OCCUPATION_TYPE"].value_counts()

In [ ]:
plt.figure(figsize=(10, 8))

sns.barplot(data=df,
            y=df["OCCUPATION_TYPE"].value_counts().index,
            x=df["OCCUPATION_TYPE"].value_counts().values,
            palette='Set2')

In [ ]:
df["STATUS"].value_counts()

In [ ]:
plt.figure(figsize=(10, 8))

sns.barplot(data=df,
            y=df["STATUS"].value_counts().index,
            x=df["STATUS"].value_counts().values,
            palette='rocket')

In [ ]:
df['STATUS'].replace("C" , "Good_Debt" , inplace = True)
df['STATUS'].replace("X" , "Good_Debt" , inplace = True)
df['STATUS'].replace("0" , "Good_Debt" , inplace = True)
df['STATUS'].replace("1" , "Good_Debt" , inplace = True)
df['STATUS'].replace("2" , "Bad_Debt" , inplace = True)
df['STATUS'].replace("3" , "Bad_Debt" , inplace = True)
df['STATUS'].replace("4" , "Bad_Debt" , inplace = True)
df['STATUS'].replace("5" , "Bad_Debt" , inplace = True)

In [ ]:
df["STATUS"].value_counts()

In [ ]:
sns.countplot (x='STATUS',data=df)

**imbalance data**

In [ ]:
Numerical=df.select_dtypes(exclude='object').columns.tolist()
Numerical

In [ ]:
value_count_dict = dict()
for i in Numerical :
    print(i,'column\'s count values : ' )
    index , count = df[i].value_counts().index , df[i].value_counts().values
    value_count_dict['value']  = list(index)
    value_count_dict['count'] = list(count)
    value_count_df = pd.DataFrame(value_count_dict)
    if value_count_df.shape[0]<10:
        display(value_count_df.head(10))
    else :
        display(value_count_df.head())

In [ ]:
fig, axes = plt.subplots(1,3)

g1=sns.countplot(x=df.FLAG_PHONE,linewidth=1, ax=axes[0],palette="Set2")
g1.set_title("Phone status")
g1.set_xlabel("Count")

g2=sns.countplot(x=df.FLAG_EMAIL,linewidth=1.2, ax=axes[1],palette="Set2")
g2.set_title("Email Status")
g2.set_xlabel("Count")

g3=sns.countplot(x=df.FLAG_WORK_PHONE,linewidth=1.2, ax=axes[2],palette="Set2")
g3.set_title("FLAG_WORK_PHONE Status")
g3.set_xlabel("Count")

fig.set_size_inches(14,5)

plt.tight_layout()


plt.show()

In [ ]:
df["CNT_CHILDREN"].value_counts()

In [ ]:

sns.boxplot( x=df["CNT_CHILDREN"])

In [ ]:
df["CNT_FAM_MEMBERS"].value_counts()

In [ ]:
sns.boxplot( x=df["CNT_FAM_MEMBERS"],palette="husl")

In [ ]:
df["AMT_INCOME_TOTAL"].agg(["min" , 'max' , 'mean' , "std"])

In [ ]:
fig = px.scatter(df, x="AMT_INCOME_TOTAL")
fig.show()

In [ ]:
df["DAYS_BIRTH"]

In [ ]:
df["DAYS_BIRTH"].agg(["min" , 'max'])

In [ ]:


df['DAYS_BIRTH'] = datetime.strptime("2024-01-01", "%Y-%m-%d") + df['DAYS_BIRTH'].apply(pd.offsets.Day)

In [ ]:
df['DAYS_BIRTH']

In [ ]:
plt.hist(df['DAYS_BIRTH'])
plt.xlabel('Year')
plt.ylabel('Frequency')
plt.title('Distribution of Birth Years')
plt.show()

In [ ]:
df["DAYS_EMPLOYED"]

In [ ]:
df["DAYS_EMPLOYED"].agg(["min" , 'max'])

In [ ]:

df['DAYS_EMPLOYED'] = datetime.strptime("2024-01-01", "%Y-%m-%d") + df['DAYS_EMPLOYED'].apply(pd.offsets.Day)

In [ ]:
df['DAYS_EMPLOYED'] 

In [ ]:
df["DAYS_EMPLOYED"].agg(["min" , 'max'])

In [ ]:
plt.hist(df['DAYS_EMPLOYED'])
plt.xlabel('Year')
plt.ylabel('Frequency')
plt.title('Distribution of Employed Years')
plt.show()

**right skweness**

In [ ]:
df["MONTHS_BALANCE"]

In [ ]:
df["MONTHS_BALANCE"]=df["MONTHS_BALANCE"]*-1

In [ ]:
df["MONTHS_BALANCE"]

In [ ]:
df["MONTHS_BALANCE"].agg(["min" , 'max'])

In [ ]:
ax = df["MONTHS_BALANCE"].hist( # Remove grid
                        xlabelsize=10, # Change size of labels on the x-axis
                        ylabelsize=12, # Change size of labels on the y-axis
                        bins=20, # Number of bins
                        edgecolor='black', # Color of the border
                        color='orange', # Color of the bins
                        rwidth=0.8 # Space between bins
                       )

# Add a bold title ('\n' allow us to jump rows)
ax.set_title('Distribution of \nthe months length',
             weight='bold') 

# Add label names
ax.set_xlabel('MONTHS_BALANCE')
ax.set_ylabel('Frequency')

# Show the plot
plt.show()

***left skweness*******

In [ ]:
df.info()

# Preprocessing

visualization clear to us some issues about data like:
*   some coulumns have imbalanced like status and email_status
*   some coulumns have oultiers like AMT_INCOME_TOTAL,CNT_FAM_MEMBERS and CNT_CHILDREN
*   coulums have skweness like DAYS_EMPLOYED
*    normalize data


**handling outliers**

In [ ]:
# Select the columns where you want to remove outliers
columns_to_filter = ['AMT_INCOME_TOTAL', 'CNT_FAM_MEMBERS', 'CNT_CHILDREN']

# Calculate lower and upper bounds for each selected column
lower_bounds = df[columns_to_filter].quantile(0.05)
upper_bounds = df[columns_to_filter].quantile(0.90)

# Define the boundaries for outlier removal
lower_bound_limits = lower_bounds - 0.05 * (upper_bounds - lower_bounds)
upper_bound_limits = upper_bounds + 0.05 * (upper_bounds - lower_bounds)

# Filter out the rows containing outliers in any of the selected columns
df = df[~((df[columns_to_filter] < lower_bound_limits) | (df[columns_to_filter] > upper_bound_limits)).any(axis=1)]


In [ ]:
df.info()

In [ ]:
sns.boxplot( data=df[["AMT_INCOME_TOTAL"]],palette="husl")

In [ ]:
sns.boxplot( data=df[["CNT_FAM_MEMBERS","CNT_CHILDREN"]],palette="husl")

#### label_encoding

In [ ]:
lst = list(df.select_dtypes('object').columns)

 

for i in lst :

  lb = LabelEncoder()

  df[i] = lb.fit_transform(df[i])

In [ ]:
df=df.drop(["FLAG_PHONE"], axis=1)

In [ ]:
x = df.drop(["ID",'STATUS'], axis=1)
y = df['STATUS']

### oversampling data

In [ ]:


# Assuming x and y are your feature and target variables respectively
smote = SMOTE()
X_balance, Y_balance = smote.fit_resample(x, y)


In [ ]:

x_train, x_test, y_train, y_test = train_test_split(X_balance, Y_balance, test_size=0.2, random_state=16)


In [ ]:
# Skew function of Pandas
old_skew = df.skew().sort_values(ascending=False)
old_skew

#### handling skewness 

In [ ]:
power = PowerTransformer(method='yeo-johnson')
x_train = power.fit_transform(x_train)
x_test = power.fit_transform(x_test)

In [ ]:
# normalizer = StandardScaler()
# x_train = normalizer.fit_transform(x_train)
# x_test = normalizer.fit_transform(x_test)

### Modeling & evaluation
* LogisticRegression
* RandomForestClassifier
* XGBClassifier


**LogisticRegression**

In [ ]:
log_model = LogisticRegression()
log_model.fit(x_train, y_train)
print('Logistic Model  Train Accuracy : ',log_model.score(x_train, y_train)*100, '%')
print('Logistic Model Test Accuracy : ', log_model.score(x_test, y_test)*100, '%')

prediction = log_model.predict(x_test)
      


In [ ]:
print('\nClassification report:')      
print(classification_report(y_test, prediction))



In [ ]:

conf_matrix = confusion_matrix(y_test, prediction)

# Plot confusion matrix
plt.figure(figsize=(2, 2))
sns.set_style('white')
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix: Logistic Regression')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Assuming model2 is your trained classification model
# Assuming x_test and y_test are your test features and labels, respectively

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, log_model.predict_proba(x_test)[:, 1])

# Calculate AUC
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

**RandomForestClassifier**

In [ ]:
model = RandomForestClassifier(random_state = 0)
model.fit(x_train, y_train)
print("RandomForest Train Accuracy = ",model.score(x_train, y_train))
y_predict = model.predict(x_test)
print("RandomForest Test Accuracy = " , accuracy_score(y_test, y_predict))
print("F1 = " , f1_score(y_test, y_predict))



In [ ]:
      
print('\nClassification report:')      
print(classification_report(y_test, y_predict))

In [ ]:

conf_matrix = confusion_matrix(y_test, y_predict)

# Plot confusion matrix
plt.figure(figsize=(2, 2))
sns.set_style('white')
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Purples', cbar=False)
plt.title('Confusion Matrix: Logistic Regression')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Assuming model2 is your trained classification model
# Assuming x_test and y_test are your test features and labels, respectively

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(x_test)[:, 1])

# Calculate AUC
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


**XGBoost**

In [ ]:
model2 =  XGBClassifier()
model2.fit(x_train, y_train)
print(" XGB Train Accuracy = " ,model2.score(x_train, y_train))
y_predict = model.predict(x_test)
print(" XGB Test Accuracy = " , accuracy_score(y_test, y_predict))
print("F1 = " , f1_score(y_test, y_predict))

      


In [ ]:
print('\nClassification report:')      
print(classification_report(y_test, y_predict))

In [ ]:

# Plot confusion matrix
plt.figure(figsize=(2,2))
sns.set_style('white')
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='PuBuGn_r', cbar=False)
plt.title('Confusion Matrix: Logistic Regression')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Assuming model2 is your trained classification model
# Assuming x_test and y_test are your test features and labels, respectively

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, model2.predict_proba(x_test)[:, 1])

# Calculate AUC
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
feat_importances = model.feature_importances_

# Create a pandas Series with feature importances
feat_importances_series = pd.Series(feat_importances, index=df.columns[1:-1])

# Sort feature importances in descending order
feat_importances_series = feat_importances_series.sort_values(ascending=False)

# Plot feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x=feat_importances_series.values, y=feat_importances_series.index, palette='viridis')
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importance Plot')
plt.show()

In [ ]:
# pip install pycaret


In [ ]:
# from pycaret.regression import *
# from pycaret.classification import *